In [1]:
!cp -r "/content/drive/MyDrive/New_Labels.zip" "/content/New_Labels.zip"
!cp -r "/content/drive/MyDrive/New_masks.zip"  "/content/New_masks.zip"
!unzip "/content/New_masks.zip"
!unzip "/content/New_Labels.zip"
!git clone https://github.com/HRNet/HRNet-Semantic-Segmentation.git /content/HRNet

Streaming output truncated to the last 5000 lines.
  inflating: New_masks/207/fibula left/fibula left1_mask.npy  
  inflating: __MACOSX/New_masks/207/fibula left/._fibula left1_mask.npy  
  inflating: New_masks/405/femur left/femur left1_mask.npy  
  inflating: __MACOSX/New_masks/405/femur left/._femur left1_mask.npy  
  inflating: New_masks/405/pelvis left/pelvis left1_mask.npy  
  inflating: __MACOSX/New_masks/405/pelvis left/._pelvis left1_mask.npy  
  inflating: New_masks/253/fibula right/fibula right1_mask.npy  
  inflating: __MACOSX/New_masks/253/fibula right/._fibula right1_mask.npy  
  inflating: New_masks/253/no image/no image1_mask.npy  
  inflating: __MACOSX/New_masks/253/no image/._no image1_mask.npy  
  inflating: New_masks/253/tibia right/tibia right1_mask.npy  
  inflating: __MACOSX/New_masks/253/tibia right/._tibia right1_mask.npy  
  inflating: New_masks/253/femur right/femur right1_mask.npy  
  inflating: __MACOSX/New_masks/253/femur right/._femur right1_mask.npy  
  

In [2]:
import os
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any, Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
!pip install yacs

In [3]:
import os
from PIL import Image

IMAGE_EXTS = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp'}

def is_image(path):
    if os.path.splitext(path)[1].lower() not in IMAGE_EXTS:
        return False
    try:
        with Image.open(path) as img:
            img.verify()
        return True
    except Exception:
        return False

import cv2
import numpy as np
import os
from glob import glob

images_folder_path = "/content/New_Labels"
masks_folder_path  = "/content/New_masks"

X = []
Y = []

def first_image_in_folder(folder):
    """Return first image path in folder, or None."""
    for entry in os.scandir(folder):
        if entry.is_file() and is_image(entry.path):
            return entry.path
    return None

count = 0
for subfolder in os.listdir(images_folder_path):

    img_subdir = os.path.join(images_folder_path, subfolder)
    mask_subdir = os.path.join(masks_folder_path, subfolder)

    if not os.path.isdir(img_subdir):
        continue

    count+=1

    print("Processing:", count)

    img_paths = [entry.path for entry in os.scandir(img_subdir)
                 if entry.is_file() and is_image(entry.path)]

    for p in img_paths:
        X.append(cv2.imread(p))

    if len(img_paths) == 0:
        continue

    ref_img = cv2.imread(img_paths[0])
    H, W = ref_img.shape[:2]

    labels_tensor = np.zeros((H, W), dtype=bool)

    mask_paths = glob(os.path.join(mask_subdir, "**/*.npy"), recursive=True)

    for mp in mask_paths:
        m = np.load(mp)
        labels_tensor |= (m.astype(bool))

    Y.append(labels_tensor)


Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5
Processing: 6
Processing: 7
Processing: 8
Processing: 9
Processing: 10
Processing: 11
Processing: 12
Processing: 13
Processing: 14
Processing: 15
Processing: 16
Processing: 17
Processing: 18
Processing: 19
Processing: 20
Processing: 21
Processing: 22
Processing: 23
Processing: 24
Processing: 25
Processing: 26
Processing: 27
Processing: 28
Processing: 29
Processing: 30
Processing: 31
Processing: 32
Processing: 33
Processing: 34
Processing: 35
Processing: 36
Processing: 37
Processing: 38
Processing: 39
Processing: 40
Processing: 41
Processing: 42
Processing: 43
Processing: 44
Processing: 45
Processing: 46
Processing: 47
Processing: 48
Processing: 49
Processing: 50
Processing: 51
Processing: 52
Processing: 53
Processing: 54
Processing: 55
Processing: 56
Processing: 57
Processing: 58
Processing: 59
Processing: 60
Processing: 61
Processing: 62
Processing: 63
Processing: 64
Processing: 65
Processing: 66
Processing: 67
Proc

In [4]:
import os
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader

def get_fold_dataloaders(
    items,
    fold_number,
    train_tf=None,
    val_tf=None,
    batch_size=3,
    num_workers=3,
    seed=42,
):
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    for fold, (train_idx, val_idx) in enumerate(kf.split(items), start=1):
        if fold == fold_number:
            print(f"Reconstructing Fold {fold_number}...")

            train_items = [items[i] for i in train_idx]
            val_items   = [items[i] for i in val_idx]

            train_ds = BinarySegDataset(train_items, transform=train_tf)
            val_ds   = BinarySegDataset(val_items,   transform=val_tf)

            train_loader = DataLoader(
                train_ds,
                batch_size=batch_size,
                shuffle=True,
                num_workers=num_workers,
                pin_memory=True,
                collate_fn=pad_collate_fn
            )

            val_loader = DataLoader(
                val_ds,
                batch_size=batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=True,
                collate_fn=pad_collate_fn
            )

            return train_loader, val_loader

    raise ValueError("Invalid fold number: expected 1–5")

@torch.no_grad()
def infer_and_save(
    model,
    val_loader,
    output_dir,
    threshold=0.5,
    device="cuda"
):
    os.makedirs(output_dir, exist_ok=True)
    input_dir  = os.path.join(output_dir, "inputs")
    mask_dir   = os.path.join(output_dir, "masks")
    prob_dir   = os.path.join(output_dir, "probs")

    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    os.makedirs(prob_dir, exist_ok=True)

    device = torch.device(device)
    model.to(device).eval()

    global_idx = 0

    for batch_idx, (images, masks, sizes) in enumerate(val_loader):
        images = images.to(device)

        logits = model(images)
        if logits.dim() == 3:
            logits = logits.unsqueeze(1)

        probs = torch.sigmoid(logits)

        for i, (H, W) in enumerate(sizes):
            img_tensor = images[i, :, :H, :W].cpu()
            prob_i = probs[i, 0, :H, :W].cpu().numpy()
            pred_mask = (prob_i >= threshold).astype(np.uint8)

            img_np = img_tensor.numpy()
            img_np = img_np - img_np.min()
            img_np = img_np / (img_np.max() + 1e-7)
            img_np = (img_np * 255).astype(np.uint8)

            if img_np.shape[0] == 1:
                img_np = img_np[0]
            else:
                img_np = np.transpose(img_np, (1, 2, 0))

            Image.fromarray(img_np).save(
                os.path.join(input_dir, f"val_{global_idx}_input.png")
            )

            Image.fromarray(pred_mask * 255).save(
                os.path.join(mask_dir, f"val_{global_idx}_mask.png")
            )

            prob_uint8 = (prob_i * 255).astype(np.uint8)
            Image.fromarray(prob_uint8).save(
                os.path.join(prob_dir, f"val_{global_idx}_prob.png")
            )

            global_idx += 1

    print(f"Saved {global_idx} validation predictions to {output_dir}")

def run_fold3_inference(
    items,
    fold3_ckpt_path,
    output_dir,
    build_model_fn,
    train_tf=None,
    val_tf=None,
    batch_size=3,
    num_workers=3,
    seed=42,
    threshold=0.5,
):
    _, fold3_val_loader = get_fold_dataloaders(
        items=items,
        fold_number=3,
        train_tf=train_tf,
        val_tf=val_tf,
        batch_size=batch_size,
        num_workers=num_workers,
        seed=seed
    )

    model = build_model_fn()

    print(f"Loading fold 3 best checkpoint: {fold3_ckpt_path}")
    ckpt = torch.load(fold3_ckpt_path, map_location="cpu")
    sd = ckpt.get("model", ckpt)
    sd = {k.replace("module.", ""): v for k, v in sd.items()}
    model.load_state_dict(sd, strict=False)

    infer_and_save(
        model=model,
        val_loader=fold3_val_loader,
        output_dir=output_dir,
        threshold=threshold,
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

In [5]:
def load_hrnet_backbone_only(model, ckpt_path):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    sd = ckpt.get("state_dict", ckpt.get("model", ckpt))
    sd = {k.replace("module.", ""): v for k, v in sd.items()}
    for k in ["last_layer.3.weight", "last_layer.3.bias"]:
        sd.pop(k, None)
    return model.load_state_dict(sd, strict=False)


In [8]:
import sys
sys.path.append("/content/HRNet/lib")
sys.path.append("/content/HRNet")

import numpy as np
if not hasattr(np, "int"):   np.int   = int
if not hasattr(np, "bool"):  np.bool  = bool
if not hasattr(np, "float"): np.float = float

from argparse import Namespace
from config import config, update_config
from models.seg_hrnet import HighResolutionNet

import torch
import torch.nn as nn

cfg_file = "/content/seg_hrnet_w18_binary_512x512.yaml"
args = Namespace(cfg=cfg_file, opts=[])
update_config(config, args)

model = HighResolutionNet(config)

old_conv = model.last_layer[3]
model.last_layer[3] = nn.Conv2d(
    old_conv.in_channels, 1, kernel_size=1, stride=1, padding=0
)

pretrained = "/content/HRNet_W18_C_ssld_pretrained.pth"
ckpt = torch.load(pretrained, map_location="cpu")

state_dict = ckpt.get("state_dict", ckpt.get("model", ckpt))
state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

head_keys = ["last_layer.3.weight", "last_layer.3.bias"]
for k in head_keys:
    state_dict.pop(k, None)

missing, unexpected = model.load_state_dict(state_dict, strict=False)

print("Loaded pretrained backbone.")
print("Missing keys (expected):", missing)
print("Unexpected keys:", unexpected)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model ready on", device)


Loaded pretrained backbone.
Missing keys (expected): ['last_layer.0.weight', 'last_layer.0.bias', 'last_layer.1.weight', 'last_layer.1.bias', 'last_layer.1.running_mean', 'last_layer.1.running_var', 'last_layer.3.weight', 'last_layer.3.bias']
Unexpected keys: ['incre_modules.0.0.conv1.weight', 'incre_modules.0.0.bn1.weight', 'incre_modules.0.0.bn1.bias', 'incre_modules.0.0.bn1.running_mean', 'incre_modules.0.0.bn1.running_var', 'incre_modules.0.0.conv2.weight', 'incre_modules.0.0.bn2.weight', 'incre_modules.0.0.bn2.bias', 'incre_modules.0.0.bn2.running_mean', 'incre_modules.0.0.bn2.running_var', 'incre_modules.0.0.conv3.weight', 'incre_modules.0.0.bn3.weight', 'incre_modules.0.0.bn3.bias', 'incre_modules.0.0.bn3.running_mean', 'incre_modules.0.0.bn3.running_var', 'incre_modules.0.0.downsample.0.weight', 'incre_modules.0.0.downsample.1.weight', 'incre_modules.0.0.downsample.1.bias', 'incre_modules.0.0.downsample.1.running_mean', 'incre_modules.0.0.downsample.1.running_var', 'incre_modul

In [12]:
from sklearn.model_selection import KFold
import torch
torch.cuda.empty_cache()

assert len(X) == len(Y)

items = list(zip(X, Y))

cfg = TrainConfig(device="cuda", epochs=40, lr=1e-3, threshold=0.5)

def build_model_fn():
    m = HighResolutionNet(config)
    return m

train_tf = None
val_tf   = None

fold_results = run_5fold_cv(
    items=items,
    build_model_fn=build_model_fn,
    ckpt_path="/content/HRNet_W18_C_ssld_pretrained.pth",
    save_root="/content/runs_h18_5fold",
    cfg=cfg,
    train_tf=train_tf,
    val_tf=val_tf,
    batch_size=3,
    num_workers=3,
    seed=42
)

print("Fold results (best_loss, best_dice, best_hd95):")
for i, (loss, dice, hd) in enumerate(fold_results, start=1):
    print(f"Fold {i}: loss={loss:.4f}, dice={dice:.4f}, hd95={hd:.4f}")



========== Fold 1/5 ==========


/tmp/ipython-input-220032540.py:291: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=cfg.amp and device.type == "cuda")
/tmp/ipython-input-220032540.py:315: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=cfg.amp and device.type == "cuda"):


Epoch 1/40 | iter 20/134 | loss 0.9508
Epoch 1/40 | iter 40/134 | loss 0.6612
Epoch 1/40 | iter 60/134 | loss 0.5769
Epoch 1/40 | iter 80/134 | loss 0.5642
Epoch 1/40 | iter 100/134 | loss 0.5662
Epoch 1/40 | iter 120/134 | loss 0.5039
Epoch 1 val_loss=0.5533 | Dice=0.6717 | HD95=63.5974
Epoch 2/40 | iter 20/134 | loss 0.5496
Epoch 2/40 | iter 40/134 | loss 0.5114
Epoch 2/40 | iter 60/134 | loss 0.4661
Epoch 2/40 | iter 80/134 | loss 0.5162
Epoch 2/40 | iter 100/134 | loss 0.5063
Epoch 2/40 | iter 120/134 | loss 0.5138
Epoch 2 val_loss=0.4764 | Dice=0.7384 | HD95=65.0782
Epoch 3/40 | iter 20/134 | loss 0.4506
Epoch 3/40 | iter 40/134 | loss 0.4685
Epoch 3/40 | iter 60/134 | loss 0.4560
Epoch 3/40 | iter 80/134 | loss 0.5177
Epoch 3/40 | iter 100/134 | loss 0.4621
Epoch 3/40 | iter 120/134 | loss 0.5717
Epoch 3 val_loss=0.4550 | Dice=0.7456 | HD95=61.2306
Epoch 4/40 | iter 20/134 | loss 0.4388
Epoch 4/40 | iter 40/134 | loss 0.5017
Epoch 4/40 | iter 60/134 | loss 0.4503
Epoch 4/40 | ite

In [15]:
import os
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader

def get_fold_dataloaders(
    items,
    fold_number,
    train_tf=None,
    val_tf=None,
    batch_size=3,
    num_workers=3,
    seed=42,
):
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    for fold, (train_idx, val_idx) in enumerate(kf.split(items), start=1):
        if fold == fold_number:
            print(f"Reconstructing Fold {fold_number}...")

            train_items = [items[i] for i in train_idx]
            val_items   = [items[i] for i in val_idx]

            train_ds = BinarySegDataset(train_items, transform=train_tf)
            val_ds   = BinarySegDataset(val_items,   transform=val_tf)

            train_loader = DataLoader(
                train_ds,
                batch_size=batch_size,
                shuffle=True,
                num_workers=num_workers,
                pin_memory=True,
                collate_fn=pad_collate_fn
            )

            val_loader = DataLoader(
                val_ds,
                batch_size=batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=True,
                collate_fn=pad_collate_fn
            )

            return train_loader, val_loader

    raise ValueError("Invalid fold number: expected 1–5")

@torch.no_grad()
def infer_and_save(
    model,
    val_loader,
    output_dir,
    threshold=0.5,
    device="cuda"
):
    os.makedirs(output_dir, exist_ok=True)
    input_dir  = os.path.join(output_dir, "inputs")
    mask_dir   = os.path.join(output_dir, "masks")
    prob_dir   = os.path.join(output_dir, "probs")

    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    os.makedirs(prob_dir, exist_ok=True)

    device = torch.device(device)
    model.to(device).eval()

    global_idx = 0

    for batch_idx, (images, masks, sizes) in enumerate(val_loader):
        images = images.to(device)

        logits = model(images)
        if logits.dim() == 3:
            logits = logits.unsqueeze(1)

        probs = torch.sigmoid(logits)

        for i, (H, W) in enumerate(sizes):
            img_tensor = images[i, :, :H, :W].cpu()
            prob_i = probs[i, 0, :H, :W].cpu().numpy()
            pred_mask = (prob_i >= threshold).astype(np.uint8)

            img_np = img_tensor.numpy()
            img_np = img_np - img_np.min()
            img_np = img_np / (img_np.max() + 1e-7)
            img_np = (img_np * 255).astype(np.uint8)

            if img_np.shape[0] == 1:
                img_np = img_np[0]
            else:
                img_np = np.transpose(img_np, (1, 2, 0))

            Image.fromarray(img_np).save(
                os.path.join(input_dir, f"val_{global_idx}_input.png")
            )

            Image.fromarray(pred_mask * 255).save(
                os.path.join(mask_dir, f"val_{global_idx}_mask.png")
            )

            prob_uint8 = (prob_i * 255).astype(np.uint8)
            Image.fromarray(prob_uint8).save(
                os.path.join(prob_dir, f"val_{global_idx}_prob.png")
            )

            global_idx += 1

    print(f"Saved {global_idx} validation predictions to {output_dir}")

def run_fold_inference(
    items,
    fold3_ckpt_path,
    output_dir,
    build_model_fn,
    train_tf=None,
    val_tf=None,
    batch_size=3,
    num_workers=3,
    seed=42,
    threshold=0.5,
):
    _, fold3_val_loader = get_fold_dataloaders(
        items=items,
        fold_number=3,
        train_tf=train_tf,
        val_tf=val_tf,
        batch_size=batch_size,
        num_workers=num_workers,
        seed=seed
    )

    model = build_model_fn()

    print(f"Loading fold 3 best checkpoint: {fold3_ckpt_path}")
    ckpt = torch.load(fold3_ckpt_path, map_location="cpu")
    sd = ckpt.get("model", ckpt)
    sd = {k.replace("module.", ""): v for k, v in sd.items()}
    model.load_state_dict(sd, strict=False)

    infer_and_save(
        model=model,
        val_loader=fold3_val_loader,
        output_dir=output_dir,
        threshold=threshold,
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

items = list(zip(X, Y))

def build_model_fn():
     return HighResolutionNet(config)

run_fold_inference(
     items=items,
     fold3_ckpt_path="/content/runs_h18_5fold/fold_3/best.pt",
     output_dir="/content/fold3_outputs",
     build_model_fn=build_model_fn,
     train_tf=None,
     val_tf=None,
     batch_size=3,
     num_workers=3,
     seed=42,
     threshold=0.5
)




Reconstructing Fold 3...
Loading fold 3 best checkpoint: /content/runs_h18_5fold/fold_3/best.pt
Saved 100 validation predictions to /content/fold3_outputs


In [16]:
import shutil

shutil.make_archive("val_preds", "zip", "fold3_outputs")

'/content/my_zipped_folder.zip'